<a href="https://colab.research.google.com/github/sauravsingla/General/blob/master/Customer_Dynamic_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/content"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

train = pd.read_csv("train.tsv", sep='\t')
test = pd.read_csv("test.tsv", sep='\t')




=conda3
Data-Science
sample_data
sample_submission.csv
test.tsv
train.tsv



In [ ]:
# new features
# len of description

train['item_description'] = train['item_description'].astype(str)
test['item_description'] = test['item_description'].astype(str)

train['des_len'] = train['item_description'].apply(lambda x: len(x))
test['des_len'] = test['item_description'].apply(lambda x: len(x))

In [ ]:
# words in description
train['word_count'] = train['item_description'].apply(lambda x: len(x.split()))
test['word_count'] = test['item_description'].apply(lambda x: len(x.split()))

In [ ]:
# men len of words in description inversed and scaled
train['mean_des'] = train['item_description'].apply(lambda x: float(len(x.split())) / len(x))  * 10
test['mean_des'] = test['item_description'].apply(lambda x: float(len(x.split())) / len(x)) * 10 


In [ ]:
# length of name
train['name_len'] = train['name'].apply(lambda x: len(x))
test['name_len'] = test['name'].apply(lambda x: len(x))

In [ ]:
# words in name
train['word_name'] = train['name'].apply(lambda x: len(x.split()))
test['word_name'] = test['name'].apply(lambda x: len(x.split()))

In [ ]:
# men len of words in name inversed and scaled
train['mean_name'] = train['name'].apply(lambda x: float(len(x.split())) / len(x))  * 10
test['mean_name'] = test['name'].apply(lambda x: float(len(x.split())) / len(x)) * 10 

In [ ]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,des_len,word_count,mean_des,name_len,word_name,mean_name
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,18,3,1.666667,35,7,2.000000
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,188,36,1.914894,32,4,1.250000
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,124,29,2.338710,14,2,1.428571
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,173,32,1.849711,21,3,1.428571
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,41,5,1.219512,20,4,2.000000


In [ ]:
# missing value imputation
train.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          0
des_len                   0
word_count                0
mean_des                  0
name_len                  0
word_name                 0
mean_name                 0
dtype: int64

In [ ]:
train['category_name'].value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings                 60177
Women/Tops & Blouses/T-Shirts                                  46380
Beauty/Makeup/Face                                             34335
Beauty/Makeup/Lips                                             29910
Electronics/Video Games & Consoles/Games                       26557
Beauty/Makeup/Eyes                                             25215
Electronics/Cell Phones & Accessories/Cases, Covers & Skins    24676
Women/Underwear/Bras                                           21274
Women/Tops & Blouses/Blouse                                    20284
Women/Tops & Blouses/Tank, Cami                                20284
Women/Dresses/Above Knee, Mini                                 20082
Women/Jewelry/Necklaces                                        19758
Women/Athletic Apparel/Shorts                                  19528
Beauty/Makeup/Makeup Palettes                                  19103
Women/Shoes/Boots                 

In [ ]:
train['category_name'].fillna('ppp/ppp/ppp', inplace=True)
test['category_name'].fillna('ppp/ppp/ppp', inplace=True)

In [ ]:
#train[ train['category_name'].str.contains('Electronics') ]['price'].mean()

In [ ]:
train['elec'] = train['category_name'].apply(lambda x : int('electronics' in x.lower()))
test['elec'] = test['category_name'].apply(lambda x : int('electronics' in x.lower()))

In [ ]:
train['category_name'].value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings                 60177
Women/Tops & Blouses/T-Shirts                                  46380
Beauty/Makeup/Face                                             34335
Beauty/Makeup/Lips                                             29910
Electronics/Video Games & Consoles/Games                       26557
Beauty/Makeup/Eyes                                             25215
Electronics/Cell Phones & Accessories/Cases, Covers & Skins    24676
Women/Underwear/Bras                                           21274
Women/Tops & Blouses/Blouse                                    20284
Women/Tops & Blouses/Tank, Cami                                20284
Women/Dresses/Above Knee, Mini                                 20082
Women/Jewelry/Necklaces                                        19758
Women/Athletic Apparel/Shorts                                  19528
Beauty/Makeup/Makeup Palettes                                  19103
Women/Shoes/Boots                 

In [ ]:
train['brand_name'].fillna('ttttttt', inplace=True)
test['brand_name'].fillna('ttttttt', inplace=True)

In [ ]:
train.isnull().sum()
test.isnull().sum()

test_id              0
name                 0
item_condition_id    0
category_name        0
brand_name           0
shipping             0
item_description     0
des_len              0
word_count           0
mean_des             0
name_len             0
word_name            0
mean_name            0
elec                 0
dtype: int64

In [ ]:
# types of category
#train['cat_len'] = train['category_name'].apply(lambda x: len( x.split('/')))
#test['cat_len'] = test['category_name'].apply(lambda x: len( x.split('/')))

In [ ]:
#length of category words
train['cat_lennn'] = train['category_name'].apply(lambda x: len(x))
test['cat_lennn'] = test['category_name'].apply(lambda x: len(x))

In [ ]:
def was_priced(x):
    return int('[rm]' in x)
               
train['rm'] = train['item_description'].apply( lambda x : was_priced(x))
test['rm'] = test['item_description'].apply( lambda x : was_priced(x))
               

In [ ]:
train['was_described'] = 1
test['was_described'] = 1

train.loc[ train['item_description'] == 'No description yet','was_described'] = 0
test.loc[ test['item_description'] == 'No description yet','was_described'] = 0


In [ ]:
# description containes 'new' word
train['new'] = train['item_description'].apply(lambda x : int('new' in x.lower()))
test['new'] = test['item_description'].apply(lambda x : int('new' in x.lower()))

In [ ]:
# splitting subcategories of category_name
train_cat = pd.DataFrame(train.category_name.str.split('/',2).tolist(),
                                   columns = ['sub1','sub2', 'sub3'])
train['sub1'] = train_cat['sub1']
train['sub2'] = train_cat['sub2']
train['sub3'] = train_cat['sub3']

test_cat = pd.DataFrame(test.category_name.str.split('/',2).tolist(),
                                   columns = ['sub1','sub2', 'sub3'])

test['sub1'] = test_cat['sub1']
test['sub2'] = test_cat['sub2']
test['sub3'] = test_cat['sub3']

train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,des_len,word_count,...,word_name,mean_name,elec,cat_lennn,rm,was_described,new,sub1,sub2,sub3
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,ttttttt,10.0,1,No description yet,18,3,...,7,2.000000,0,17,0,0,0,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,188,36,...,4,1.250000,1,50,0,1,0,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,124,29,...,2,1.428571,0,27,0,1,0,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,ttttttt,35.0,1,New with tags. Leather horses. Retail for [rm]...,173,32,...,3,1.428571,0,34,1,1,1,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,ttttttt,44.0,0,Complete with certificate of authenticity,41,5,...,4,2.000000,0,23,0,1,0,Women,Jewelry,Necklaces


In [ ]:
train['hand'] = train['category_name'].apply(lambda x : int('handmade' in x.lower()))
test['hand'] = test['category_name'].apply(lambda x : int('handmade' in x.lower()))

In [ ]:
train['men'] = train['category_name'].apply(lambda x : int('men' in x.lower()))
test['men'] = test['category_name'].apply(lambda x : int('men' in x.lower()))

In [ ]:
# int in description
import re
train['int_desc'] = train['item_description'].apply(lambda x : int(bool(re.search(r'\d',x))))
test['int_desc'] = test['item_description'].apply(lambda x : int(bool(re.search(r'\d',x))))

In [ ]:
# integer was present in name
train['int_name'] = train['name'].apply(lambda x : int(bool(re.search(r'\d',x))))
test['int_name'] = test['name'].apply(lambda x : int(bool(re.search(r'\d',x))))

In [ ]:
# word condition was present in description
train['cond'] = train['item_description'].apply(lambda x : int('condition' in x.lower()))
test['cond'] = test['item_description'].apply(lambda x : int('condition' in x.lower()))

In [ ]:
train['category_name'].value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings                 60177
Women/Tops & Blouses/T-Shirts                                  46380
Beauty/Makeup/Face                                             34335
Beauty/Makeup/Lips                                             29910
Electronics/Video Games & Consoles/Games                       26557
Beauty/Makeup/Eyes                                             25215
Electronics/Cell Phones & Accessories/Cases, Covers & Skins    24676
Women/Underwear/Bras                                           21274
Women/Tops & Blouses/Blouse                                    20284
Women/Tops & Blouses/Tank, Cami                                20284
Women/Dresses/Above Knee, Mini                                 20082
Women/Jewelry/Necklaces                                        19758
Women/Athletic Apparel/Shorts                                  19528
Beauty/Makeup/Makeup Palettes                                  19103
Women/Shoes/Boots                 

In [ ]:
# converting price to log scale
positive = train['price'].values > 0
negative = train['price'].values < 0
train['price'] = np.piecewise(train['price'], (positive, negative), (np.log, lambda x: -np.log(-x)))

features = ['int_name',  'cond','int_desc', 'new', 'was_described', 'men', 'rm', 'item_condition_id','cat_lennn',  'brand_name', 'shipping', 'des_len', 'name_len','mean_des', 'word_count', 'mean_name', 'word_name', 'sub1', 'sub2', 'hand', 'elec', 'category_name']

data = train[features]
data_sub = test[features]

y = train['price']

In [ ]:
data_sub.head()

,int_name,cond,int_desc,new,was_described,men,rm,item_condition_id,cat_lennn,brand_name,...,name_len,mean_des,word_count,mean_name,word_name,sub1,sub2,hand,elec,category_name
0,0,0,1,0,1,1,0,1,19,ttttttt,...,40,3.333333,2,2.000000,8,Women,Jewelry,0,0,Women/Jewelry/Rings
1,1,0,1,1,1,0,0,1,39,ttttttt,...,40,1.513944,38,1.750000,7,Other,Office supplies,0,0,Other/Office supplies/Shipping Supplies
2,0,0,0,1,1,0,1,1,46,Coach,...,9,2.000000,11,2.222222,2,Vintage & Collectibles,Bags and Purses,0,0,Vintage & Collectibles/Bags and Purses/Handbag
3,0,0,0,0,1,1,0,2,23,ttttttt,...,13,1.492537,10,1.538462,2,Women,Sweaters,0,0,Women/Sweaters/Cardigan
4,0,1,1,0,1,0,0,3,35,ttttttt,...,16,1.736527,29,1.875000,3,Other,Books,0,0,Other/Books/Religion & Spirituality


In [ ]:
#label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

frames = [ data, data_sub ]
xx = pd.concat(frames)


l = [ 'brand_name', 'sub1', 'sub2', 'category_name']
for x in l :
    le.fit(xx[x])
    data[x] = le.transform(data[x])
    data_sub[x] = le.transform(data_sub[x])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [ ]:
data.head()

,int_name,cond,int_desc,new,was_described,men,rm,item_condition_id,cat_lennn,brand_name,...,name_len,mean_des,word_count,mean_name,word_name,sub1,sub2,hand,elec,category_name
0,0,0,0,0,0,1,0,3,17,5287,...,35,1.666667,3,2.000000,7,5,102,0,0,829
1,0,1,0,0,1,0,0,3,50,3889,...,32,1.914894,36,1.250000,4,1,30,0,1,86
2,0,0,1,0,1,1,0,1,27,4588,...,14,2.338710,29,1.428571,2,9,103,0,0,1277
3,0,0,0,1,1,0,1,1,34,5287,...,21,1.849711,32,1.428571,3,3,55,0,0,503
4,1,0,0,0,1,1,0,1,23,5287,...,20,1.219512,5,2.000000,4,9,58,0,0,1204


In [ ]:
from sklearn import ensemble
clf =  ensemble.GradientBoostingRegressor( learning_rate = 0.7, n_estimators=700, max_depth = 3,warm_start = True, verbose=1, random_state=45, max_features = 0.8)
clf.fit(data, y)


      Iter       Train Loss   Remaining Time 
         1           0.5617           39.85m
         2           0.5379           33.58m
         3           0.5214           35.66m
         4           0.5097           36.54m
         5           0.5012           36.49m
         6           0.4948           35.92m
         7           0.4906           36.12m
         8           0.4848           35.97m
         9           0.4769           36.02m
        10           0.4735           35.53m
        20           0.4459           34.18m
        30           0.4304           34.25m
        40           0.4168           33.58m
        50           0.4077           32.97m
        60           0.4014           32.45m
        70           0.3965           32.00m
        80           0.3918           31.88m
        90           0.3858           31.29m
       100           0.3829           30.61m
       200           0.3612           25.52m
       300           0.3521           20.41m
       40

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.7, loss='ls', max_depth=3, max_features=0.8,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=700, n_iter_no_change=None, presort='auto',
             random_state=45, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=1, warm_start=True)

In [ ]:
# predicting and saving to output file
predicted = clf.predict(data_sub) 

print(features)
print( clf.feature_importances_)

['int_name', 'cond', 'int_desc', 'new', 'was_described', 'men', 'rm', 'item_condition_id', 'cat_lennn', 'brand_name', 'shipping', 'des_len', 'name_len', 'mean_des', 'word_count', 'mean_name', 'word_name', 'sub1', 'sub2', 'hand', 'elec', 'category_name']
[1.21425748e-02 1.51071094e-03 9.47084349e-03 2.02006613e-02
 9.84158956e-05 4.44973978e-02 4.00737919e-03 3.85779749e-02
 1.01997116e-01 2.98555480e-01 1.27958535e-01 1.63569552e-02
 7.29400793e-03 3.51337056e-03 3.74266217e-03 4.10779488e-03
 2.07102367e-03 1.14642938e-02 9.33911369e-02 4.18536641e-03
 1.54286314e-02 1.79427667e-01]


In [ ]:
out = pd.DataFrame()

In [ ]:
#sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
out['test_id'] = test['test_id']
out['price'] = predicted
out['price'] = np.exp(out['price'])

In [ ]:
out.head()
out.to_csv("output5.csv",index=False)